# Data Overview

**Important**: this notebook produces not the results shown in the Stylometric Analysis section of the Project Report, but a corpus which can then be subjected to the analysis by using the [Stylo](https://github.com/computationalstylistics/stylo) library.

In [1]:
import os
import pandas as pd
from math import ceil
from nltk import word_tokenize
from collections import defaultdict

In [2]:
DATA_PATH = 'data/data.tsv'

In [3]:
dialogue_data = pd.read_table(DATA_PATH)

# Removing redundant whitespace from the text data
cleaned_text = []
for i in range(len(dialogue_data)):
    text = str(dialogue_data.iloc[i]["text"])
    cleaned_text.append(" ".join(text.split()))

cleaned_text_column = pd.DataFrame({'text': cleaned_text})

dialogue_data.update(cleaned_text_column)

In [9]:
dialogue_data.head()

,Unnamed: 0,text,character_id,character_gender,scene_id,date,play_id,genre
0,0,Mir gehts verteufelt! – Bruder! die Schuldner ...,0,MALE,0,1779,0,Tragedy
1,1,Ja! Herr Bruder! wenn soll ich sie wieder habe...,1,MALE,0,1779,0,Tragedy
2,2,O! Behalt deine paar lumpen Ducaten! Ich will ...,0,MALE,0,1779,0,Tragedy
3,3,Nu! Nu! S'is ja mein Ernst noch nicht. Hier si...,1,MALE,0,1779,0,Tragedy
4,4,"Hast du keinen Schnaps? Ich habe, hohl mich de...",0,MALE,0,1779,0,Tragedy


In [4]:
dialogue_data["genre"].unique()

array(['Tragedy', nan, 'Comedy', 'Tragicomedy', 'Libretto'], dtype=object)

In [5]:
dialogue_data = dialogue_data.dropna(subset=["genre"])

In [39]:
def get_lines_for_play_id(data, play_id=0, feature="character_gender"):
    '''
    Get a list of all the lines uttered in a play, with a certain feature attributed
    to the line/play written in front. Example:
    - "MALE: Ja! Herr Bruder!"
    - "FEMALE: Guten Tag, Herr Waker!"
    
    :param data: dataframe from which the lines are extracted
    :param play_id: The play_id of the play of which we want the dialogue lines
    :param feature: The feature that is placed in front of each dialogue line (e.g. character_name, character_gender)
    :return: List with all the lines of play with play_id in the following format: "{feature}: {text}"
    '''
    line_list = []
    dialogues_play_id = data[data['play_id'] == play_id][[feature, "text"]]
    for row in dialogues_play_id.itertuples():
        line = str(row[1]) + ": " + row[2]
        line_list.append(line)
    return line_list


In [40]:
get_lines_for_play_id(dialogue_data, 0, "character_gender")[2]

'MALE: O! Behalt deine paar lumpen Ducaten! Ich will doch wol Geld bekommen ohne dich.'

In [42]:
def stylo_oppose_corpus(data, play_ids, feature, corpus_dir):
    '''
    Divide and write dramatic data into subcorpora.

    :param data: dataframe from which the lines are extracted
    :param play_ids: all the play_ids considered for the subcorpora
    :param feature: the feature by which the subcorpora are distinguished
    :corpus_dir: the directory where the subcorpora are stored
    '''
    unique_features = data[feature].unique()
    for play_id in play_ids:
        lines = get_lines_for_play_id(data, play_id, feature)
        for line in lines:
            for unique_feat in unique_features:
                if line.startswith(unique_feat):
                    break
            line = line.replace(unique_feat+": ", "")
            with open(os.path.join(corpus_dir, unique_feat)+".txt", "a", encoding="utf-8") as fa:
                fa.write(line+' ')

In [45]:
play_ids_1720 = [0, 4, 13, 14, 15, 18, 19, 33, 36, 45, 47, 51, 56, 57, 58, 69, 75, 77, 78, 79, 89, 93, 109, 110, 111, 114, 116, 119, 120, 121, 122, 123, 125, 127, 128, 130, 132, 135, 136, 137, 139, 141, 142, 143, 144, 145, 146, 184, 185, 220, 223, 224, 227, 255, 256, 257, 258, 259, 260, 266, 274, 279, 281, 282, 283, 284, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 311, 312, 314, 317, 318, 320, 321, 322, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 361, 366, 375, 376, 377, 378, 395, 396, 397, 398, 402, 407, 417, 431, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 469, 470, 471, 472, 473, 474, 494, 495, 498, 501, 502, 504, 517, 529, 530, 533, 534, 535, 536, 537, 543, 568, 571, 572, 582, 583, 596]
play_ids_1815 = [2, 3, 5, 6, 7, 8, 9, 10, 11, 20, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 37, 38, 39, 40, 41, 44, 50, 53, 64, 65, 66, 67, 68, 70, 74, 84, 85, 86, 88, 90, 91, 94, 104, 105, 107, 115, 124, 126, 129, 133, 147, 148, 149, 150, 151, 152, 153, 154, 156, 176, 177, 178, 179, 187, 189, 203, 204, 205, 206, 208, 210, 211, 212, 213, 214, 215, 216, 217, 221, 225, 226, 239, 245, 246, 254, 261, 262, 263, 264, 265, 268, 269, 270, 278, 280, 287, 296, 310, 313, 323, 324, 327, 328, 329, 330, 331, 332, 356, 360, 363, 364, 367, 372, 373, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 399, 400, 401, 409, 410, 411, 412, 413, 414, 415, 416, 418, 419, 420, 422, 424, 432, 441, 446, 497, 499, 500, 503, 505, 519, 520, 521, 538, 539, 540, 541, 542, 546, 548, 551, 552, 555, 560, 561, 563, 567, 573, 574, 575, 576, 577, 578, 580, 585, 586, 587, 588, 589, 594, 595, 597]
play_ids_1900 = [1, 17, 22, 23, 46, 62, 63, 71, 92, 95, 96, 100, 131, 190, 191, 192, 194, 196, 197, 233, 234, 237, 252, 253, 315, 316, 325, 326, 333, 334, 335, 365, 379, 408, 428, 433, 435, 439, 440, 445, 447, 449, 450, 451, 452, 477, 483, 489, 508, 515, 516, 518, 522, 523, 524, 525, 526, 527, 528, 556, 557, 558, 562, 564]

In [123]:
stylo_oppose_corpus(play_ids_1720, "character_gender", "data-stylo/oppose-gender/1720-1814/")
stylo_oppose_corpus(play_ids_1815, "character_gender", "data-stylo/oppose-gender/1815-1899/")
stylo_oppose_corpus(play_ids_1900, "character_gender", "data-stylo/oppose-gender/1900-present/")

In [29]:
stylo_oppose_corpus(play_ids_1720, "genre", "data-stylo/oppose-genre/1720-1814/")
stylo_oppose_corpus(play_ids_1815, "genre", "data-stylo/oppose-genre/1815-1899/")
stylo_oppose_corpus(play_ids_1900, "genre", "data-stylo/oppose-genre/1900-present/")

To replicate the experiment, all is needed is to create folders corresponding to the subcorpora which one then can compare using the `oppose()` function from the R library [Stylo](https://github.com/computationalstylistics/stylo).